In [2]:
import pandas as pd
Data = pd.read_csv('C:/Users/muneeba/Desktop/THESIS/DataDistributionPCA.csv')
Data=Data.sort_values(['Sensor ID'],ascending=[True])
Data1=Data.iloc[:,4:68]
Data1.head()
returns = Data1[[key for key in dict(Data1.dtypes) if dict(Data1.dtypes)[key] in ['float64', 'int64']]] 
returns = returns[:64]
returns.head()
#slicing the required data out of complete dataset
returns_arr = returns.as_matrix()[:64]
returns_arr.shape
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
returns_arr_scaled = scaler.fit_transform(returns_arr)
 

Set up the TensorFlow code for an Autoencoder
The number of inputs of the auto-encoder is equal to the number of outputs (number of columns in our input data)
We have one hidden layer with 3 neurons
Use an AdamOptimizer to minimize the loss
Notice that:
The reconstruction loss of the output is the mean-square error
Each layer has no activation function i.e is a linear layer
An Autoencoder set up this way simply performs a PCA

In [3]:
import tensorflow as tf

n_inputs = 64
n_hidden = 3  # codings
n_outputs = n_inputs

learning_rate = 0.01 #how much to train 
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])
hidden = tf.layers.dense(X, n_hidden)
outputs = tf.layers.dense(hidden, n_outputs)

reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(reconstruction_loss)

init = tf.global_variables_initializer()




Run the training data through the Autoencoder
We do not use any labels for this training only the input data
The output is the reconstructed value using only the 3 principal components
Note that the reconstruction will not be equal to the input data

In [4]:
n_iterations = 10000

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: returns_arr_scaled})
    writer = tf.summary.FileWriter('./temp', sess.graph)
    outputs_val = outputs.eval(feed_dict={X: returns_arr_scaled})
    print(outputs_val)

[[ 0.98040682  0.99364817  1.02573681 ...,  0.81256473  0.62767929
   0.44613647]
 [ 0.6797722   0.69068623  0.71737629 ...,  0.75497282  0.60723722
   0.46216038]
 [-0.31247959 -0.30916569 -0.30005044 ...,  0.57260543  0.54688966
   0.52154511]
 ..., 
 [-0.21827975 -0.21476582 -0.19601636 ...,  0.16171642  0.22193621
   0.26965544]
 [ 0.98809898  0.99974483  1.05642486 ..., -0.51198232 -0.39647359
  -0.31798971]
 [ 1.20264792  1.21568251  1.2788527  ..., -0.63386345 -0.5089348
  -0.42525342]]


In [5]:
import numpy as np
data=scaler.inverse_transform(outputs_val,copy=None)
#recovered
#data
orig=np.linalg.norm(outputs_val,ord=1)
DiffData=data-returns_arr
NewNorm=np.linalg.norm(DiffData,ord=1)
error=NewNorm/orig 
error
#recovering the orignal dat

0.00018632269415328148

In [6]:
#building stacked autoencoders
n_inputs = 64
n_hidden1 = 14
n_hidden2 = 3 # codings
n_hidden3 = n_hidden1
n_outputs = n_inputs

In [7]:
he_init = tf.contrib.layers.variance_scaling_initializer()
l2_regularizer = tf.contrib.layers.l2_regularizer(0.0001)

Instructions for updating:
Use the retry module or similar alternatives.


In [8]:
X = tf.placeholder(tf.float32, shape=[None, n_inputs])

In [9]:
from functools import partial

dense_layer = partial(tf.layers.dense,
                      activation=tf.nn.elu,
                      kernel_initializer=he_init,
                      kernel_regularizer=l2_regularizer)

hidden1 = dense_layer(X, n_hidden1)
hidden2 = dense_layer(hidden1, n_hidden2)
hidden3 = dense_layer(hidden2, n_hidden3)
outputs = dense_layer(hidden3, n_outputs, activation=None)
print(hidden2)

Tensor("dense_3/Elu:0", shape=(?, 3), dtype=float32)


In [10]:
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_losses)

optimizer = tf.train.AdamOptimizer(0.01)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [11]:
saver = tf.train.Saver()

In [12]:
n_iterations = 10000

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        training_op.run(feed_dict={X: returns_arr_scaled})
    writer = tf.summary.FileWriter('./temp', sess.graph)
    outputs_val = outputs.eval(feed_dict={X: returns_arr_scaled})
    print(outputs_val)

[[ 0.97932196  0.99252629  1.02435338 ...,  0.80865395  0.62405938
   0.44288248]
 [ 0.67745268  0.68834722  0.71484888 ...,  0.75381565  0.60625571
   0.4614222 ]
 [-0.31526834 -0.31197777 -0.30297613 ...,  0.57254219  0.54695767
   0.52179903]
 ..., 
 [-0.22034711 -0.21683449 -0.19816804 ...,  0.16288751  0.22296107
   0.27061096]
 [ 0.98847103  1.00021243  1.05697787 ..., -0.50809973 -0.39299572
  -0.31488043]
 [ 1.20391178  1.2170682   1.28039885 ..., -0.62970054 -0.50511885
  -0.42178565]]


In [13]:
import numpy as np
data=scaler.inverse_transform(outputs_val,copy=None)
#recovered
#data
orig=np.linalg.norm(outputs_val,ord=1)
DiffData=data-returns_arr
NewNorm=np.linalg.norm(DiffData,ord=1)
error=NewNorm/orig 
error
#recovering the orignal dat

0.00022331585682723102